In [5]:
# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import norm
import pandas as pd

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

In [2]:
%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm

In [4]:
import numpy as np
import math
from PIL import Image
import time

In [6]:
# Initializations

In [5]:
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)
magnitudeMin = float("inf")
magnitudeMax = float("-inf")
phaseMin = float("inf")
phaseMax = float("-inf")

In [6]:
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
    print(phaseMax,phaseMin)
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
     
#     print(magnitudeVals)
#     print(phaseVals)
#     print(phaseRange)
#     print(magnitudeRange)
    
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals

In [7]:
def generateSpectrogramForWave(signal):
    start_time = time.time()
    global magnitudeMin
    global magnitudeMax
    global phaseMin
    global phaseMax
    buffer = np.zeros(int(signal.size + WINDOW_STEP - (signal.size % WINDOW_STEP)))
    buffer[0:len(signal)] = signal
    height = int(FFT_LENGTH / 2 + 1)
    width = int(len(buffer) / (WINDOW_STEP) - 1)
    magnitudePixels = np.zeros((height, width))
    phasePixels = np.zeros((height, width))

    for w in range(width):
        buff = np.zeros(FFT_LENGTH)
        stepBuff = buffer[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH]
        # apply hanning window
        stepBuff = stepBuff * np.hanning(WINDOW_LENGTH)
        buff[0:len(stepBuff)] = stepBuff
        #buff now contains windowed signal with step length and padded with zeroes to the end
        fft = np.fft.rfft(buff)
        for h in range(len(fft)):
            #print(buff.shape)
            #return
            magnitude = math.sqrt(fft[h].real**2 + fft[h].imag**2)
            if magnitude > magnitudeMax:
                magnitudeMax = magnitude 
            if magnitude < magnitudeMin:
                magnitudeMin = magnitude 

            phase = math.atan2(fft[h].imag, fft[h].real)
            if phase > phaseMax:
                phaseMax = phase
            if phase < phaseMin:
                phaseMin = phase
            magnitudePixels[height-h-1,w] = magnitude
            phasePixels[height-h-1,w] = phase
    rgbArray = generateLinearScale(magnitudePixels, phasePixels,
                                  magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    
    elapsed_time = time.time() - start_time
    print('%.2f' % elapsed_time, 's', sep='')
    img = Image.fromarray(rgbArray, 'RGB')
    return img

In [8]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

phaseMax -inf phaseMin inf magnitudeMax -inf magnitudeMin inf


In [40]:
path_train = './train'
path_test = './test'
path_ytrain = './y_train'
path_ytest = './y_test'

In [41]:
path_WAVA = './WAVA'
path_WAVA = './WAVB'

In [42]:
import random
import shutil

random.seed(42)

In [8]:
# Create 1 sec wav files and store it in *.npy format

In [1]:
for filename in tqdm(glob.glob(os.path.join(path_train, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
#     img.save(filename[:-4]+'.png')
    np.save(filename[:-4]+'.npy', img) # save

In [44]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

phaseMax 3.141592653589793 phaseMin -3.141592653589793 magnitudeMax 2211683.973249525 magnitudeMin 0.0


In [2]:
for filename in tqdm(glob.glob(os.path.join(path_test, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
    np.save(filename[:-4]+'.npy', img) # save

In [46]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

phaseMax 3.141592653589793 phaseMin -3.141592653589793 magnitudeMax 2211683.973249525 magnitudeMin 0.0


In [3]:
for filename in tqdm(glob.glob(os.path.join(path_ytrain, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
    np.save(filename[:-4]+'.npy', img) # save

In [48]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

phaseMax 3.141592653589793 phaseMin -3.141592653589793 magnitudeMax 5432292.907520762 magnitudeMin 0.0


In [4]:
for filename in tqdm(glob.glob(os.path.join(path_ytest, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
    np.save(filename[:-4]+'.npy', img) # save

In [50]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

phaseMax 3.141592653589793 phaseMin -3.141592653589793 magnitudeMax 5432292.907520762 magnitudeMin 0.0


In [51]:
path_mat_train = './mat_train'
path_mat_test = './mat_test'
path_mat_ytrain = './mat_ytrain'
path_mat_ytest = './mat_ytest'

In [65]:
for filename in glob.glob(os.path.join(path_train, '*.npy')):
    shutil.copy(filename, path_mat_train)
    os.remove(filename)
for filename in glob.glob(os.path.join(path_ytrain, '*.npy')):
    shutil.copy(filename, path_mat_ytrain)
    os.remove(filename) 

    
for filename in glob.glob(os.path.join(path_test, '*.npy')):
    shutil.copy(filename, path_mat_test)
    os.remove(filename)    
for filename in glob.glob(os.path.join(path_ytest, '*.npy')):
    shutil.copy(filename, path_mat_ytest)
    os.remove(filename)

In [10]:
# Use below initialization values for recovery

In [11]:
phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 2211683.973249525
magnitudeMin = 0.0

rate = 16000

In [61]:
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
        
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered

In [12]:
# Test the recovery - convert npy to wav

In [63]:
filename = path_mat_test+'/'+'19-198-0034.npy'
new_num_arr = np.load(filename)
new_num_arr.shape
data = recoverSignalFromSpectrogram(new_num_arr)
scipy.io.wavfile.write('./'+filename[:-4]+'.wav', rate, data)

3.141592653589793 -3.141592653589793
